In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
from df_tokenizer import DfTokenizer
from l_calculator import LCalculator
from m_calculator import MCalculator
from n_calculator import NCalculator
from q_calculator import QCalculator
from r_calculator import RCalculator
from d_calculator import DCalculator
from c_calculator import CCalculator

In [3]:
files = pd.read_csv('files.csv', encoding='utf-8')

In [4]:
files_tokenizer = DfTokenizer(files, 'file')

In [5]:
requirements = pd.read_csv('requirements.csv', encoding='utf-8')

In [6]:
requirements_tokenizer = DfTokenizer(requirements, 'id')

In [7]:
complectations_file = open('complectations.json', 'r', encoding='utf-8')
complectations_data = complectations_file.read()
complectations_file.close()

In [8]:
tracer_file = open('tracer.json', 'r', encoding='utf-8')
tracer_data = tracer_file.read()
tracer_file.close()

In [9]:
dependencies_file = open('dependencies.json', 'r', encoding='utf-8')
dependencies_data = dependencies_file.read()
dependencies_file.close()

In [10]:
complectations = json.loads(complectations_data)
tracer = json.loads(tracer_data)
dependencies = json.loads(dependencies_data)

In [11]:
# Получить константы l, m, n

In [12]:
n_calculator = NCalculator(requirements)
m_calculator = MCalculator(files)
q_calculator = QCalculator(tracer, files_tokenizer, requirements_tokenizer, n_calculator, m_calculator)
l_calculator = LCalculator(complectations)
r_calculator = RCalculator(complectations, requirements_tokenizer, n_calculator)
d_calculator = DCalculator(dependencies, files_tokenizer, m_calculator)
c_calculator = CCalculator(n_calculator)

In [13]:
n = n_calculator.calculate()
m = m_calculator.calculate()
l = l_calculator.calculate()
Q = q_calculator.calculate()
R = r_calculator.calculate()
D = d_calculator.calculate()
C = c_calculator.calculate()

In [23]:
constants_dict = {
    'n': n,
    'm': m,
    'l': l
}
constants_json = json.dumps(constants_dict)

In [25]:
with open('out/constants.json', 'w', encoding='utf-8') as constaints_file:
    constaints_file.write(constants_json)

In [32]:
Q_df = pd.DataFrame(Q)
Q_df.to_csv('out/Q.csv')

In [33]:
R_df = pd.DataFrame(R)
R_df.to_csv('out/R.csv')

In [34]:
D_df = pd.DataFrame(D)
D_df.to_csv('out/D.csv')

In [35]:
C_df = pd.DataFrame(C)
C_df.to_csv('out/C.csv')